In [1]:
import pandas as pd

In [3]:
#df = pd.read_csv(f'data/pbp/raw/pbp_query_20{sea_1}20{sea_2}.csv',low_memory=False) 
df = pd.read_csv(f'data/pbp/raw/pbp_query_20202021.csv',low_memory=False) 
df = df[df.event_type.isin(['SHOT','MISS','GOAL'])].copy()
df.columns

Index(['season', 'game_id', 'game_date', 'session', 'event_index',
       'game_period', 'game_seconds', 'clock_time', 'event_type',
       'event_description', 'event_detail', 'event_zone', 'event_team',
       'event_player_1', 'event_player_2', 'event_player_3', 'event_length',
       'coords_x', 'coords_y', 'num_on', 'num_off', 'players_on',
       'players_off', 'home_on_1', 'home_on_2', 'home_on_3', 'home_on_4',
       'home_on_5', 'home_on_6', 'home_on_7', 'away_on_1', 'away_on_2',
       'away_on_3', 'away_on_4', 'away_on_5', 'away_on_6', 'away_on_7',
       'home_goalie', 'away_goalie', 'home_team', 'away_team', 'home_skaters',
       'away_skaters', 'home_score', 'away_score', 'game_score_state',
       'game_strength_state', 'home_zone', 'pbp_distance', 'event_distance',
       'event_angle', 'home_zonestart', 'face_index', 'pen_index',
       'shift_index', 'pred_goal'],
      dtype='object')

In [8]:
teams = df.home_team.unique().tolist()
adj_shot_dict = {}
for team in teams:
    df_temp = df[df.home_team == team]
    average = round(df_temp.event_distance.mean(),2)
    adj_shot_dict[team] = average

In [28]:
adj_shot_dict['S.J']

35.4

In [12]:
def make_adjustment(df):
    team = df.home_team
    distance = df.event_distance
    adj_dist = distance - adj_shot_dict[team]
    return adj_dist

In [19]:
#df['adj_dist'] = df['event_distance'].apply(make_adjustment)

In [15]:
df['adj_dist'] = df.groupby('home_team')['event_distance'] \
                   .apply(lambda x: x - x.mean())

In [31]:
df[['event_distance','adj_dist','adj_dist_2','home_team','away_team','event_team']]

,event_distance,adj_dist,adj_dist_2,home_team,away_team,event_team
1,32.65,-6.534938,-6.53,PHI,PIT,PHI
4,47.17,7.985062,7.99,PHI,PIT,PIT
7,48.27,9.085062,9.09,PHI,PIT,PHI
19,60.31,21.125062,21.13,PHI,PIT,PIT
22,54.20,15.015062,15.02,PHI,PIT,PHI
...,...,...,...,...,...,...
250344,24.08,-11.323500,-11.32,S.J,ARI,S.J
250346,25.24,-10.163500,-10.16,S.J,ARI,S.J
250349,34.79,-0.613500,-0.61,S.J,ARI,ARI
250351,17.09,-18.313500,-18.31,S.J,ARI,ARI


In [29]:
df['adj_dist_2'] = df['event_distance'] - df['home_team'].map(adj_shot_dict)

In [32]:
import time
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [ ]:
X = .iloc[:,:-1].values
y = .iloc[:,-1].values